In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [5]:
terms = {
        'BonusMalus': {'type': 'ns'}, 
        'VehAge': {'type': 'ns'},
        'DrivAge': {'type': 'ns'}, 
        'VehPower': {'type': 'ns'}, 
        'Area': {'type': 'target_encoding'},
        'Region': {'type': 'target_encoding'},
        'VehGas': {'type': 'target_encoding'},
        'VehBrand': {'type': 'target_encoding'}
    }

interactions=[{
        "VehPower": {"type": "linear"},
        "Region": {"type": "categorical"},
        "target_encoding": True,
        "include_main": False
    }]

result = rs.glm_dict(
    response='ClaimCount',
    terms=terms,
    interactions=interactions,
    data=data,
    family='poisson',
    offset ='Exposure'
).fit(store_design_matrix=False)

In [6]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(BonusMalus, df=10) + ns(VehAge, df=10) + ns(DrivAge, df=10) + ns(VehPower, df=10) + TE(Area) + TE(Region) + TE(VehGas) + TE(VehBrand)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 42, 'df_resid': 406397, 'converged': True, 'iterations': 7}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21278.12, loss=0.309507, deviance=125795.54, log_likelihood=-83671.59, aic=167427.19, gini=0.2842, auc=0.6421, ae_ratio=1.0061, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1925.66, actual=0.388437, predicted=0.075252, ae_ratio=5.1618), DecileMetrics(decile=2, n=40643, exposure=4239.91, actual=0.22524, predicted=0.088722, ae_ratio=2.5387), DecileMetrics(decile=3, n=40643, exposure=8285.61, actual=0.163537, predicted=0.084447, ae_ratio=1.9366), DecileMetrics(decile=4, n=40643, exposure=14050.16, actual=0.119643, 

In [7]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        678012
Link Function:       (default)       Df Residuals:            677970
Method:              IRLS + Smooth   Df Model:                    41
Scale:               1.0000          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                   7
Non-zero coefs:      41             

Log-Likelihood:         -139170.7309 Deviance:                209341.3156
AIC:                     278425.4618 Null Deviance:           220244.0296
BIC:                     278905.3924 Pearson chi2:             1522773.18
Converged:           True           

------------------------------------------------------------------------------
Variable                  Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept              -3.0416 

In [6]:
result.predict(holdout_data)

array([0.0516742 , 0.04900786, 0.04947089, ..., 0.00558624, 0.01767213,
       0.07128406], shape=(135957,))

In [7]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "Region": {"type": "target_encoding"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()


In [8]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406428
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   6

Log-Likelihood:          -84120.8546 Deviance:                126694.0594
AIC:                     168263.7091 Null Deviance:           132328.3090
BIC:                     168383.7762 Pearson chi2:              934688.11
Converged:           True           

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -4.4571     0.1617  -27.566  <0.0001   [ -4.7740,  -4.1402]  ***
BonusMalus             0.0219     0.0004   54.572  

In [9]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [10]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21477.13, loss=0.311688, deviance=126682.33, log_likelihood=-84114.99, aic=168251.98, gini=0.2597, auc=0.6298, ae_ratio=0.9968, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1903.71, actual=0.412879, predicted=0.078185, ae_ratio=5.2808), DecileMetrics(decile=2, n=40643, exposure=4176.47, actual=0.236803, predicted=0.091684, ae_ratio=2.5828), DecileMetrics(decile=3, n=40643, exposure=8110.45, actual=0.164356, predicted=0.087767, ae_ratio=1.8726), DecileMetrics(decile=4, n=40643, exposure=13733.34, actual=0.127864, predicted=0.0833, ae_ratio=1.535), DecileMetrics(decile=5

In [11]:
predictions = result.predict(holdout_data)

In [12]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [13]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [14]:
predictions2 = loaded.predict(holdout_data)

In [15]:
import numpy as np
print(np.array_equal(predictions2, predictions))

True


In [16]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21477.13, loss=0.311688, deviance=126682.33, log_likelihood=-84114.99, aic=168251.98, gini=0.2597, auc=0.6298, ae_ratio=0.9968, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1903.71, actual=0.412879, predicted=0.078185, ae_ratio=5.2808), DecileMetrics(decile=2, n=40643, exposure=4176.47, actual=0.236803, predicted=0.091684, ae_ratio=2.5828), DecileMetrics(decile=3, n=40643, exposure=8110.45, actual=0.164356, predicted=0.087767, ae_ratio=1.8726), DecileMetrics(decile=4, n=40643, exposure=13733.34, actual=0.127864, predicted=0.0833, ae_ratio=1.535), DecileMetrics(decile=5

In [17]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [18]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(Region) + BonusMalus + I(BonusMalus ** 2)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 4, 'df_resid': 406435, 'converged': True, 'iterations': 6, 'regularization': {'type': 'elastic_net', 'alpha': 0.065836, 'l1_ratio': 0.5, 'cv_deviance': 0.318789, 'cv_folds': 5, 'selection': 'min'}}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21372.92, loss=0.318662, deviance=129516.61, log_likelihood=-85532.13, aic=171072.26, gini=0.1823, auc=0.5911, ae_ratio=1.0016, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1695.09, actual=0.640675, predicted=0.088769, ae_ratio=7.2173), DecileMetrics(decile=2, n=40643, exposure=3749.41, actual=0.287779, predicted=0.098946, ae_ratio=2.9084), DecileMetrics(decile=3, n=40643, exposure=7532.64, actual=0.182539, predicted=0.094688, ae_ratio=1.9278), DecileMetrics(decile=4

In [19]:
result3 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "BonusMalus": {"type": "ns"},
        "VehAge": {"type": "ns"},
        "DrivAge": {"type": "linear"},
        "VehBrand": {"type": "target_encoding"}
    },
    interactions= [
        {
            "VehAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        },
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ],
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net",
    include_unregularized=False
)

In [20]:
import rustystats as rs

# Model spec that caused the issue when elastic_net was applied
terms = {
    "BonusMalus": {'type' : "linear"},
    "VehAge": {'type' : "ns"}, 
    "DrivAge": {'type' : "ns"},
    "Area": {'type' : "target_encoding"},
}

interactions= [
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ]

# Build the model
glm = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,  # freMTPL2freq dataset
    family="poisson",
    offset="Exposure",  # log-linked
)

# This is where the issue occurs - regularized fit + diagnostics
result = glm.fit()

# Check if diagnostics returns None for deviance
diag = result.diagnostics(train_data=train_data, test_data=holdout_data)
print(diag.to_dict())  # Check train_test.test.deviance

{'model_summary': {'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=10) + ns(DrivAge, df=10) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 22, 'df_resid': 406417, 'converged': True, 'iterations': 7}, 'train_test': {'train': {'dataset': 'train', 'n_obs': 406439, 'total_exposure': 214930.9, 'total_actual': 21408.0, 'total_predicted': 21440.04, 'loss': 0.310818, 'deviance': 126328.41, 'log_likelihood': -83938.03, 'aic': 167920.06, 'gini': 0.2737, 'auc': 0.6368, 'ae_ratio': 0.9985, 'ae_by_decile': [{'decile': 1, 'n': 40643, 'exposure': 1903.65, 'actual': 0.397133, 'predicted': 0.077483, 'ae_ratio': 5.1254}, {'decile': 2, 'n': 40643, 'exposure': 4158.7, 'actual': 0.240219, 'predicted': 0.090959, 'ae_ratio': 2.641}, {'decile': 3, 'n': 40643, 'exposure': 8130.91, 'actual': 0.164557, 'predicted': 0.08705, 'ae_ratio': 1.8904}, {'decile': 4, 'n': 40643, 'exposure': 13710.21, 'actual': 0.12363, 'predicted': 0.082758, 'ae_ratio': 1.4939}, {'decile': 5, 'n': 40643